In [2]:
# Exemploes de data frames com spark 

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exemplo DataFrame").getOrCreate() # inicializa o spark


df = spark.createDataFrame([("João", 10),("Maria", 20),("Pedro", 30)])
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/07 21:11:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/07 21:11:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
                                                                                

+-----+---+
|   _1| _2|
+-----+---+
| João| 10|
|Maria| 20|
|Pedro| 30|
+-----+---+



In [3]:
# Exemplo DF com schema 

schema = "ID int, Nome string"
dados = [[1, "João"], [2, "Maria"]]

df2 = spark.createDataFrame(dados, schema)
df2.show()


+---+-----+
| ID| Nome|
+---+-----+
|  1| João|
|  2|Maria|
+---+-----+



In [6]:
# Exemplo de operação sum no data frame
from pyspark.sql.functions import sum

schema2 = "produto string, venda int"
dados = [["caneta", 10],["lapis", 10],["caneta", 40]]

df3 = spark.createDataFrame(dados, schema2)
df3.show()

agrupado = df3.groupBy("produto").agg(sum("venda"))
agrupado.show()


+-------+-----+
|produto|venda|
+-------+-----+
| caneta|   10|
|  lapis|   10|
| caneta|   40|
+-------+-----+

+-------+----------+
|produto|sum(venda)|
+-------+----------+
| caneta|        50|
|  lapis|        10|
+-------+----------+



In [7]:
# Exemplo select para seleção de colunas quando necessário

df3.select("venda", "produto").show()

+-----+-------+
|venda|produto|
+-----+-------+
|   10| caneta|
|   10|  lapis|
|   40| caneta|
+-----+-------+



In [9]:
# Exemplo de expressões 
from pyspark.sql.functions import expr

df3.select("venda", "produto", expr("venda * 2")).show()

+-----+-------+-----------+
|venda|produto|(venda * 2)|
+-----+-------+-----------+
|   10| caneta|         20|
|   10|  lapis|         20|
|   40| caneta|         80|
+-----+-------+-----------+

